In [107]:
# -*- coding: utf-8 -*-

from __future__ import division, unicode_literals

import pandas as pd
import numpy as np

import dateutil
import datetime
import time
from time import sleep

from bs4 import BeautifulSoup
import requests

import re
import html5lib

import pickle as pk

import ast

In [167]:
# Selenium imports
import selenium.selenium

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pyvirtualdisplay import Display

import os

In [4]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print f.__name__, 'took', end - start, 'seconds'
        return result
    return f_timer

In [224]:
# Setting up chromedriver
chromedriver = '/usr/bin/'
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome()

For each URL:
1. Get Petition ID
2. Get Comments (Click down 100 times)
3. If fail to get comments, log "No Comments"

In [351]:
@timefunc
def click_100_buttons(driver):
    try:
        load_more = driver.find_element_by_xpath('//button[@class="btn btn-full btn-big"]')
        for i in range(100):
            try:
                load_more.click()
                driver.execute_script("window.scrollTo(0,1000000000);")
                sleep(.1)
            except:
                break
    except:
        pass
    sleep(1)

In [335]:
@timefunc
def get_comments(url, driver):
    d_comm = {}
    driver.get(url)
    
    click_100_buttons(driver)
    
    try:
        petition_holder = driver.find_element_by_xpath('//div[@data-view="petitions/comments/index"]')
        petition_id = ast.literal_eval(petition_holder.get_attribute('data-fetch_summary'))['petition']['id']
        
        try: 
            comments_holder = driver.find_element_by_xpath('//div[@class="js-show-comments-top-rated"]')
            comments = comments_holder.find_elements_by_xpath('.//div[@data-view="components/comments_feed/comment_card"]')

            for comment in comments:
                comment_id = ast.literal_eval(comment.get_attribute('data-fetch_summary'))['model']['id']
                if comment_id in d_comm:
                    pass
                else:
                    comment_text = comment.find_element_by_xpath('.//p[@class="type-break-word mbxs"]').text
                    commenter_name = comment.find_element_by_xpath('.//div[@class="col-xs-8"]/strong').text
                    commenter_location = comment.find_element_by_xpath('.//div[@class="col-xs-8"]/span[@class="type-weak"]').text[2:]
                    comment_date = comment.find_element_by_xpath('.//div[@class="col-xs-4 txt-r"]').text
                    comment_hearts = comment.find_element_by_xpath('.//div[@class="type-s mvxxs"]').text

                    d_comm[comment_id] = {'comment_text': comment_text,
                                         'commenter_name': commenter_name,
                                         'commenter_location': commenter_location,
                                         'comment_date': comment_date,
                                         'comment_hearts': comment_hearts}
        except:
            return petition_id, "No Comments"
        return petition_id, d_comm
    except:
        pass       

For each URL pickle:
1. Open URLs
2. For each URL
    3. Write d_pet to pickle (if success)
    4. Write fails to pickle (if fail)

In [338]:
@timefunc
def get_multiple_url_comments(urls, driver):
    d_pet = {}
    fails = []
    for url in urls:
        try:
            petition_id, d_comm = get_comments(url, driver)
            d_pet[petition_id] = d_comm
        except:
            fails.append(url)
    return d_pet, fails

@timefunc
def get_all_comments(url_pickle, driver):
    with open(url_pickle, 'rb') as f:
        urls = pk.load(f)
        comment_urls = [url + '/c' for url in urls]
    d_pet, fails = get_multiple_url_comments(comment_urls, driver)
    with open(url_pickle[:-4] + '_comments.pkl', 'wb') as f:
        pk.dump(d_pet,f,-1)
    with open(url_pickle[:-4] + '_fails.pkl', 'wb') as f:
        pk.dump(fails,f,-1)
    return d_pet, fails

In [339]:
test, fails = get_all_comments('urls100.pkl', driver)

click_100_buttons took 1.01685500145 seconds
get_comments took 4.86172199249 seconds
click_100_buttons took 1.00937509537 seconds
get_comments took 3.47912788391 seconds
click_100_buttons took 3.6055290699 seconds
get_comments took 14.9379141331 seconds
click_100_buttons took 2.46226787567 seconds
get_comments took 9.18500685692 seconds
click_100_buttons took 1.61735606194 seconds
get_comments took 6.09054923058 seconds
click_100_buttons took 18.3410978317 seconds
get_comments took 71.8046550751 seconds
click_100_buttons took 5.15780997276 seconds
get_comments took 18.7589042187 seconds
click_100_buttons took 1.00825715065 seconds
get_comments took 3.69798207283 seconds
click_100_buttons took 1.74126195908 seconds
get_comments took 6.39542007446 seconds
click_100_buttons took 1.00816583633 seconds
get_comments took 3.91502189636 seconds
click_100_buttons took 2.49735808372 seconds
get_comments took 4.45005989075 seconds
click_100_buttons took 7.97765493393 seconds
get_comments took 26.

In [353]:
@timefunc
def get_all_comments_and_pickle(url_pickle, driver):
    with open(url_pickle, 'rb') as f:
        urls = pk.load(f)
        comment_urls = [url + '/c' for url in urls]
    
    d_pet = {}
    fails = []
    
    for url in comment_urls:
        try:
            petition_id, d_comm = get_comments(url, driver)
            d_pet[petition_id] = d_comm
            print "Entries: ", len(d_pet)
            with open(url_pickle[:-4] + '_comments.pkl', 'wb') as f:
                pk.dump(d_pet,f,-1)
        except:
            fails.append(url)
            print "Fails: ", len(fails)
            with open(url_pickle[:-4] + '_fails.pkl', 'wb') as f:
                pk.dump(fails,f,-1)
    
    return d_pet, fails

In [355]:
test1000, fails1000 = get_all_comments_and_pickle('urls1000.pkl', driver)

click_100_buttons took 1.03306889534 seconds
get_comments took 4.79798197746 seconds
Entries:  1
click_100_buttons took 1.00858902931 seconds
get_comments took 3.23129487038 seconds
Entries:  2
click_100_buttons took 5.62904405594 seconds
get_comments took 22.308825016 seconds
Entries:  3
click_100_buttons took 2.79947209358 seconds
get_comments took 9.34239196777 seconds
Entries:  4
click_100_buttons took 2.10013699532 seconds
get_comments took 7.83704805374 seconds
Entries:  5
click_100_buttons took 4.20969319344 seconds
get_comments took 16.2709829807 seconds
Entries:  6
click_100_buttons took 4.86597895622 seconds
get_comments took 16.6784751415 seconds
Entries:  7
click_100_buttons took 1.00730490685 seconds
get_comments took 3.64713597298 seconds
Entries:  8
click_100_buttons took 1.89614081383 seconds
get_comments took 6.53469991684 seconds
Entries:  9
click_100_buttons took 1.01951503754 seconds
get_comments took 4.15970611572 seconds
Entries:  10
click_100_buttons took 2.49951